In [1]:
import numpy as np
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
from tqdm import tqdm
import random

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ngram_range = 2
max_features = 20000
maxlen = 50
batch_size = 64
embedded_size = 128
epoch = 10

In [3]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [534, 2516, 3188, 18774, 36, 7790, 219, 150, 19, 4274] ['rock', 'destined', '21st', 'centurys', 'new', 'conan', 'hes', 'going', 'make', 'splash']


In [5]:
idx_trainset = []
for text in trainset.data:
    idx = []
    for t in text.split():
        try:
            idx.append(dictionary[t])
        except:
            pass
    idx_trainset.append(idx)

In [6]:
def create_ngram_set(input_list, ngram_value):
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))

def build_ngram(x_train):
    global max_features
    ngram_set = set()
    for input_list in tqdm(x_train, total=len(x_train), ncols=70):
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    max_features = np.max(list(indice_token.keys())) + 1
    return token_indice

def add_ngram(sequences, token_indice):
    new_sequences = []
    for input_list in tqdm(sequences, total=len(sequences), ncols=70):
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)
    return new_sequences

In [7]:
token_indice = build_ngram(idx_trainset)
X = add_ngram(idx_trainset, token_indice)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen)

100%|████████████████████████| 10662/10662 [00:00<00:00, 64394.97it/s]


In [8]:
X.shape

(10662, 50)

In [9]:
train_X, test_X, train_Y, test_Y = train_test_split(X, 
                                                    trainset.target,
                                                    test_size = 0.2)

In [10]:
class Model:
    def __init__(self, embedded_size, dict_size, dimension_output, learning_rate):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        self.logits = tf.layers.dense(tf.reduce_mean(encoder_embedded, 1), dimension_output)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=self.logits,
            labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(embedded_size,vocabulary_size+4,2,1e-3)
sess.run(tf.global_variables_initializer())

In [12]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : train_X[i:i+batch_size], model.Y : train_Y[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : test_X[i:i+batch_size], model.Y : test_Y[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.533144
time taken: 0.6158695220947266
epoch: 0, training loss: 0.691755, training acc: 0.522791, valid loss: 0.686008, valid acc: 0.533144

epoch: 1, pass acc: 0.533144, current acc: 0.579545
time taken: 0.5032393932342529
epoch: 1, training loss: 0.674360, training acc: 0.606438, valid loss: 0.673455, valid acc: 0.579545

epoch: 2, pass acc: 0.579545, current acc: 0.625947
time taken: 0.49947619438171387
epoch: 2, training loss: 0.650408, training acc: 0.687970, valid loss: 0.656426, valid acc: 0.625947

epoch: 3, pass acc: 0.625947, current acc: 0.665720
time taken: 0.4993126392364502
epoch: 3, training loss: 0.614892, training acc: 0.760221, valid loss: 0.632600, valid acc: 0.665720

epoch: 4, pass acc: 0.665720, current acc: 0.693182
time taken: 0.49781179428100586
epoch: 4, training loss: 0.566665, training acc: 0.811560, valid loss: 0.603644, valid acc: 0.693182

epoch: 5, pass acc: 0.693182, current acc: 0.721591
time taken: 0.4910674

In [13]:
logits = sess.run(model.logits, feed_dict={model.X:test_X})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.76      0.75      0.76      1060
   positive       0.76      0.76      0.76      1073

avg / total       0.76      0.76      0.76      2133

